<a href="https://colab.research.google.com/github/EvidenceN/DS-Unit-2-Kaggle-Challenge/blob/master/module3/Evidence.N%20Answers_LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Defining features and labels
train_features = 'https://raw.githubusercontent.com/EvidenceN/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/train_features.csv'
train_labels = 'https://raw.githubusercontent.com/EvidenceN/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/train_labels.csv'

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(train_features), 
                 pd.read_csv(train_labels))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('https://raw.githubusercontent.com/EvidenceN/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/test_features.csv')
sample_submission = pd.read_csv('https://raw.githubusercontent.com/EvidenceN/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/sample_submission.csv')

In [0]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [0]:
# importing necessary things. 

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

In [0]:
# creating a function to clean all data the same way. 
import numpy as np

def cleaning(x):
  """Clean train, validate, and test data sets in the same way
  
    DataFrame.copy(self, deep=True)[source]

    Make a copy of this object’s indices and data.

    When deep=True (default), a new object will be created with a copy of the 
    calling object’s data and indices. Modifications to the data or indices of 
    the copy will not be reflected in the original object (see notes below).

    When deep=False, a new object will be created without copying the calling 
    object’s data or index (only references to the data and index are copied). 
    Any changes to the data of the original will be reflected in the shallow 
    copy (and vice versa)."""

  x = x.copy()

  cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']

  for col in cols_with_zeros:
      x[col] = x[col].replace(0, np.nan)
      x[col+'_MISSING'] = x[col].isnull()

  duplicates = ['quantity_group', 'payment_type']
  x = x.drop(columns=duplicates)

  unusable_variance = ['recorded_by', 'id']
  x = x.drop(columns=unusable_variance)
    
  # Convert date_recorded to datetime
  x['date_recorded'] = pd.to_datetime(x['date_recorded'], infer_datetime_format=True)
    
  # Extract components from date_recorded, then drop the original column
  x['year_recorded'] = x['date_recorded'].dt.year
  x['month_recorded'] = x['date_recorded'].dt.month
  x['day_recorded'] = x['date_recorded'].dt.day
  x = x.drop(columns='date_recorded')
    
  # Engineer feature: how many years from construction_year to date_recorded
  x['years'] = x['year_recorded'] - x['construction_year']
  x['years_MISSING'] = x['years'].isnull()
    
  # return the wrangled dataframe
  return x

In [0]:
train = cleaning(train)
test = cleaning(test)

In [0]:
# defining mean baseline. 
mean_baseline = train['status_group'].value_counts(normalize=True)

print(f'mean baseline\n{mean_baseline}')

# defining the target and features
target = 'status_group'
features = train.columns.drop(target)

x_train = train[features]
y_train = train[target]

x_test = test[features]

mean baseline
functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64


In [0]:
from scipy.stats import randint, uniform

# creating pipeline

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42)
)

# defining the parameters to be tested when doing random search cross validation

parameters = {
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__n_estimators': randint(10, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, None],
    'randomforestclassifier__min_samples_split': range(5, 50),
    'randomforestclassifier__max_features': uniform(0,1),
}

# doing random search cv using pipeline and parameters above

searchcv = RandomizedSearchCV(
    pipeline,
    param_distributions=parameters,
    n_iter=100,
    cv = 5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-2)


In [0]:
# bringing all together. fitting my training data through my pipeline. 
# And testing different parameters using the parameters specified and the 
# randomized cross validation method. 

searchcv.fit(x_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed: 20.6min
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed: 25.0min
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed: 30.7min
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed: 34.0min
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed: 38.0min
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed: 46.7min
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed: 48.4min
[Parallel(n_jobs=-2)]: Done 167 tasks      | elapsed: 5

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            f

In [0]:
# printing out best parameters and the best score
print(f'best hyperparameters: {searchcv.best_params_}')
print(f'Validation accuracy: {searchcv.best_score_}')

best hyperparameters: {'randomforestclassifier__max_depth': 50, 'randomforestclassifier__max_features': 0.26893254140057554, 'randomforestclassifier__min_samples_split': 7, 'randomforestclassifier__n_estimators': 394, 'simpleimputer__strategy': 'mean'}
Validation accuracy: 0.8132154882154882


In [0]:
# making prediction for the test set

new_pipeline = searchcv.best_estimator_

y_pred = new_pipeline.predict(x_test)


In [0]:
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('Evidence.N DS9 Kaggle Challenge 3 - Cross Validation.csv', index=False)